In [2]:
pip install fake_useragent pandas

Note: you may need to restart the kernel to use updated packages.


In [3]:
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import pandas as pd

In [4]:
url = 'https://www.imdb.com/chart/boxoffice/?ref_=nv_ch_cht'
page = requests.get(url)
page


<Response [403]>

In [5]:
for i in range(10):
    ua = UserAgent()
    userAgent = ua.random
    headers = {'User-Agent': userAgent}
    print(headers)

{'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 13_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.0 Safari/605.1.15'}
{'User-Agent': 'Mozilla/5.0 (iPhone; CPU iPhone OS 17_3_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.0 EdgiOS/121.2277.107 Mobile/15E148 Safari/605.1.15'}
{'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'}
{'User-Agent': 'Mozilla/5.0 (Linux; Android 10; K) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Mobile Safari/537.36'}
{'User-Agent': 'Mozilla/5.0 (iPhone; CPU iPhone OS 17_0 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.0 Mobile/15E148 Safari/604.1 OPT/4.2.3'}
{'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36 Edg/121.0.0.0'}
{'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari

In [6]:
url = 'https://www.imdb.com/chart/boxoffice/?ref_=nv_ch_cht'
ua = UserAgent()
userAgent = ua.random
headers = {'User-Agent': userAgent}

page = requests.get(url, headers = headers)
soup = BeautifulSoup(page.content,'html.parser')

In [7]:
movie_box = soup.find_all('div', class_ = "ipc-metadata-list-summary-item__c")

titles = []
numbers = []
rate_votes = []

for box in movie_box:

    #titles
    if box.find('h3', class_ = "ipc-title__text") is not None:
        title = box.find('h3', class_ = "ipc-title__text")
        titles.append(title.text)
    else:
        titles.append('None')

    #Numbers
    if box.find('ul', attrs={'data-testid':'title-metadata-box-office-data-container'}) is not None:
        number = box.find('ul', attrs={'data-testid':'title-metadata-box-office-data-container'})
        numbers.append(number.text)
    else:
        numbers.append('None')

    #rate_votes
    if box.find('span', attrs={'data-testid':'ratingGroup--imdb-rating'}) is not None:
        rate_vote = box.find('span', attrs={'data-testid':'ratingGroup--imdb-rating'})
        rate_votes.append(rate_vote.text)
    else:
        rate_votes.append('None')



movie_df = pd.DataFrame({
    'title':titles,
    'number': numbers,
    'rate_vote':rate_votes
})
movie_df

,title,number,rate_vote
0,1. Venom: The Last Dance,Weekend Gross: $16MTotal Gross: $122MWeeks Rel...,6.2 (40K)
1,2. Heretic,Weekend Gross: $11MTotal Gross: $17MWeeks Rele...,7.3 (12K)
2,3. The Best Christmas Pageant Ever,Weekend Gross: $11MTotal Gross: $15MWeeks Rele...,7.4 (948)
3,4. The Wild Robot,Weekend Gross: $6.6MTotal Gross: $134MWeeks Re...,8.3 (77K)
4,5. Smile 2,Weekend Gross: $5.1MTotal Gross: $63MWeeks Rel...,7.1 (34K)
5,6. Conclave,Weekend Gross: $4.1MTotal Gross: $24MWeeks Rel...,7.6 (9.1K)
6,7. Anora,Weekend Gross: $2.5MTotal Gross: $8.7MWeeks Re...,8.2 (17K)
7,8. Here,Weekend Gross: $2.4MTotal Gross: $11MWeeks Rel...,6.3 (2.7K)
8,9. We Live in Time,Weekend Gross: $2.2MTotal Gross: $23MWeeks Rel...,7.3 (8.5K)
9,10. Terrifier 3,Weekend Gross: $1.7MTotal Gross: $54MWeeks Rel...,6.5 (31K)


In [8]:
movie_df['number'] = movie_df['number'].str.replace('Weekend Gross: ', '').str.replace('Total Gross: ', '|').str.replace('Weeks Released: ', '|')
movie_df[['weekend gross', 'total gross', 'weeks released']] = movie_df['number'].str.split('|', expand=True)

movie_df[['rating', 'vote_count']] = movie_df['rate_vote'].str.split('(', expand=True)
movie_df['vote_count'] = movie_df['vote_count'].str.replace(')','')
# # Drop the original 'rate_vote'
movie_df.drop(['number','rate_vote'], axis=1, inplace=True)
movie_df.to_csv('movie_data.csv')